In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
! which python

/home/icb/laura.martens/miniconda3/envs/cellrank/bin/python


Import packages

In [5]:
import anndata as ad
import scanpy as sc
import os
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scvelo as scv

In [6]:
import graphtools
import magic

In [7]:
scv.set_figure_params()
sc.settings.set_figure_params(dpi=100, facecolor='white', frameon=False)

In [8]:
from utils import *

In [9]:
my_cmap = get_continuous_cmap(['D6D6D6', '5D4FF4'])

## Setup paths

In [10]:
data_path = '/lustre/groups/ml01/workspace/laura.martens/moretti_colab'
save_path = os.path.join(data_path, 'biological_analysis')
scvelo_path = os.path.join(data_path, 'scvelo')

In [11]:
fig_path = '/lustre/groups/ml01/workspace/laura.martens/moretti_colab/panels'
fig_path = os.path.join(fig_path, 'epicardial')
sc.settings.figdir = fig_path

# Load data

In [12]:
adata = sc.read('/lustre/groups/ml01/workspace/laura.martens/moretti_colab/scglue/metacells.h5ad')

In [13]:
gene_act = get_act(adata)

In [14]:
act_mat = pd.DataFrame(gene_act.X.A, index=gene_act.obs_names.values, columns=gene_act.var_names.values)

In [15]:
act_mat = act_mat.fillna(0)

# Run magic on gene activities

In [54]:
magic_op = magic.MAGIC(knn=4, n_jobs=8)

In [55]:
magic_op

MAGIC(knn=4, knn_max=12, n_jobs=8)

In [20]:
g = graphtools.Graph(data=gene_act.obsm['X_glue'], knn=15, knn_max=15, decay=1, thresh=1e-4)

In [21]:
g.data = act_mat.values

In [22]:
g

,PLCXD1,GTPBP6,PPP2R3B,SHOX,CRLF2,CSF2RA,IL3RA,SLC25A6,ASMTL,P2RY8,...,SPATC1L,LSS,AP001468.1,MCM3AP,YBEY,C21orf58,PCNT,DIP2A,S100B,PRMT2
AAACCCAAGAGCAAGA-1_5,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AAACCCAAGCCGCACT-1_7,0.000000,0.0,0.000000,1.539287,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AAACCCAAGCCGTTGC-1_5,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AAACCCAAGCGAGAAA-1_6,0.758975,0.0,1.062459,0.563830,0.0,0.0,0.0,0.321138,0.922186,0.0,...,0.000000,0.563830,0.0,0.321138,0.321138,0.321138,0.563830,0.563830,0.321138,0.000000
AAACCCAAGCTCCATA-1_1,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.865161,1.321963,0.0,0.865161,0.865161,1.634358,1.634358,0.000000,0.000000,0.865161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCGGATACT-1_5,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
TTTGTTGTCGTCAAAC-1_4,0.890596,0.0,0.890596,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.890596,0.000000,0.000000,0.890596
TTTGTTGTCGTTCCCA-1_5,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
TTTGTTGTCTCTCTAA-1_6,0.596329,0.0,0.967319,0.000000,0.0,0.0,0.0,0.000000,0.967319,0.0,...,0.000000,0.000000,0.0,0.596329,0.000000,0.000000,0.000000,0.596329,0.000000,0.596329


In [58]:
imputed = magic_op.fit_transform(act_mat, genes=act_mat.columns.to_list(), graph=g)

Calculating MAGIC...
  Running MAGIC on 35499 cells and 20010 genes.
  Using precomputed graph and diffusion operator...
  Running MAGIC with `solver='exact'` on 20010-dimensional data may take a long time. Consider denoising specific genes with `genes=<list-like>` or using `solver='approximate'`.
  Calculating imputation...
  Calculated imputation in 68.75 seconds.
Calculated MAGIC in 119.05 seconds.


In [61]:
imputed

,PLCXD1,GTPBP6,PPP2R3B,SHOX,CRLF2,CSF2RA,IL3RA,SLC25A6,ASMTL,P2RY8,...,SPATC1L,LSS,AP001468.1,MCM3AP,YBEY,C21orf58,PCNT,DIP2A,S100B,PRMT2
AAACCCAAGAGCAAGA-1_5,0.055556,0.003998,0.104474,0.034883,0.032428,0.048293,0.039804,0.084153,0.166445,0.041270,...,0.055077,0.123434,0.003851,0.056885,0.063444,0.080849,0.132065,0.104130,0.013153,0.065575
AAACCCAAGCCGCACT-1_7,0.146354,0.021690,0.443581,0.072334,0.050276,0.075643,0.097663,0.207299,0.403083,0.090245,...,0.083603,0.330487,0.003814,0.174311,0.145831,0.279142,0.335252,0.417986,0.020535,0.189449
AAACCCAAGCCGTTGC-1_5,0.135491,0.019662,0.528025,0.022821,0.043418,0.118345,0.213768,0.358851,0.458120,0.066303,...,0.084860,0.428506,0.002266,0.263075,0.203587,0.357457,0.439509,0.421086,0.045937,0.218995
AAACCCAAGCGAGAAA-1_6,0.192162,0.023007,0.410781,0.166715,0.090477,0.148749,0.170198,0.246947,0.496894,0.119211,...,0.120372,0.383071,0.000000,0.260165,0.313196,0.423851,0.545100,0.400451,0.049258,0.275566
AAACCCAAGCTCCATA-1_1,0.422238,0.024819,0.318184,0.315830,0.088315,0.101957,0.162835,0.287767,0.429635,0.203998,...,0.182765,0.426673,0.004657,0.171761,0.232895,0.434658,0.597627,0.364319,0.021232,0.320016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCGGATACT-1_5,0.114193,0.016159,0.216373,0.122187,0.083275,0.063330,0.102265,0.234188,0.297853,0.047503,...,0.072305,0.163980,0.000087,0.153818,0.102500,0.274214,0.355064,0.169421,0.023392,0.196923
TTTGTTGTCGTCAAAC-1_4,0.290605,0.016466,0.434346,0.102277,0.053579,0.038486,0.145170,0.316096,0.551468,0.092917,...,0.148166,0.449649,0.000883,0.196886,0.148645,0.363220,0.559366,0.445142,0.039748,0.325480
TTTGTTGTCGTTCCCA-1_5,0.036101,0.004304,0.102782,0.016340,0.013041,0.021206,0.057825,0.110893,0.155251,0.022931,...,0.025270,0.110130,0.001148,0.049367,0.046782,0.103783,0.118857,0.102448,0.012995,0.061631
TTTGTTGTCTCTCTAA-1_6,0.146656,0.007185,0.541911,0.012165,0.046668,0.170637,0.120152,0.268419,0.461961,0.070174,...,0.112498,0.403771,0.001597,0.279460,0.184651,0.348285,0.424859,0.375739,0.032012,0.278548


In [ ]:
adata.obsm['activities_imputed'] = imputed.values

In [65]:
adata.write('/lustre/groups/ml01/workspace/laura.martens/moretti_colab/scglue/metacells_imputed.h5ad')